# 1. Import libraries and dependencies

In [1]:
import time
import pandas as pd
import json  
from pathlib import Path
import numpy as np
import time
import pyarrow.parquet as pq
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

# 2. Define input and output directories

In [2]:
# raw_data2 = Path("/export/data_ml4ds/AI4U/Datasets/ResearchPortal/20231005/rawdata/researchportal.uc3m.es/display")
# parquet_data2 = Path("/export/data_ml4ds/AI4U/Datasets/ResearchPortal/20231005/parquet")

raw_data = Path("/Users/lcsanchez/Desktop/Research/researchportal.uc3m.es/display")
parquet_data = Path("/Users/lcsanchez/Desktop/Research/researchportal.uc3m.es/parquet")

parquet_data.mkdir(parents=True, exist_ok=True)

# 3. Table `researchers`

In [8]:
all_researchers_files = list(raw_data.glob("inv*"))
all_researchers = []

for file_path in tqdm(all_researchers_files):
    if file_path.is_file():
        with file_path.open("r") as fin:
            content = fin.read()

            # Crear un objeto BeautifulSoup
            soup = BeautifulSoup(content, "html.parser")

            element = soup.find(class_="categoriainv")

            # Nombre
            try:
                element = soup.find('span', {'itemprop': 'name'})
                name = element.get_text().strip().title()
            except:
                name = None
            
            # Categoría del Investigador
            try:
                element = soup.find(class_="categoriainv")
                cat = element.get_text().split(': ')[1].strip()
            except:
                cat = None

            # ORCID
            try:
                element = soup.find(class_='individual-orcid')
                # Buscar dentro de este elemento el primer enlace y obtener su 'href'
                enlace = element.find('a')
                if enlace and 'href' in enlace.attrs:
                    orcid = enlace['href'].split('/')[-1].strip()
            except:
                orcid = None

            # Scopus
            try:
                element = soup.find(id='scopusId-noRangeClass-List')
                scopus = element.find('a').get_text().strip()
                print(scopus)
            except:
                scopus = None
                

            # Positions
            try:
                element = soup.find_all(class_='currentPosition')
                for el in element:
                    if "Academic Department" in el.get_text():
                        dep = el.find('span', itemprop='name').get_text().strip()
                    if "Research Group" in el.get_text():
                        res_group = el.find('span', itemprop='name').get_text().strip()
            except:
                dep = None
                res_group = None

            # Research Areas
            try:
                element = soup.find('ul', {'id': 'individual-hasResearchArea'})
                subject = []
                # Si se encuentra el elemento ul, extraer los textos de los elementos li
                list_items = element.find_all('li', role="listitem")
                for item in list_items:
                    subject.append(item.get_text(strip=True))
            except:
                subject = None

            # email
            try:
                element = soup.find(class_='individual-emails')
                email = element.find('a').get_text().strip()
            except:
                email = None

            # Number of publications
            try:
                element = soup.find(id='publicationsGroup').find_all('a')
                element = [el for el in element if el.get('title') == "resource name"]
                n_publis = len(element)
            except:
                n_publis = 0

            # Projects_IP (investigador principal)
            try:
                element = soup.find(id='RO_0000053-PrincipalInvestigatorRole-List').find_all('a')
                element = [el for el in element if el.get('title') == "activity name"]
                n_IP = len(element)
            except:
                n_IP = 0

            # Projects_noIP (no investigador principal)
            try:
                element = soup.find(id='RO_0000053-InvestigatorRole-List').find_all('a')
                element = [el for el in element if el.get('title') == "activity name"]
                n_noIP = len(element)
            except:
                n_noIP = 0

        all_researchers.append([file_path.stem, name, email, cat, orcid, scopus, dep, res_group, subject, n_publis, n_IP, n_noIP])

columns = ['invID', 'Name', 'Email', 'Category','ORCID', 'Scopus', 'Department', 'Research Group', 'Subjects', 'no Publis', 'Projects IP', 'Projects no IP']
df = pd.DataFrame(all_researchers, columns=columns)

  0%|▏                                         | 6/1402 [00:00<00:28, 48.97it/s]

44461256900
57226877780
57202767077
57203319680
26531865200
8637966400
6603667198


  2%|▋                                        | 22/1402 [00:00<00:20, 68.35it/s]

9639475200
35580325600
7202872762
56982928500
35073175700
56374047500
57346044900
56899987200
36150313900
6603947701
55558061000


  3%|█                                        | 36/1402 [00:00<00:26, 51.64it/s]

...

6603683704
18634592700
23034092900
6701386871
6507861147
24449439900
7103358982


 99%|██████████████████████████████████████▋| 1393/1402 [00:26<00:00, 48.32it/s]

8848712400
7101919482
36901636100
57570912600
7005982110


100%|███████████████████████████████████████| 1402/1402 [00:26<00:00, 53.11it/s]

23134288300
35324925600
55990690100
55635842100
6603908284
6506529394
7007170231


In [ ]:
print("Number of Researchers", len(df))
df.head()

In [ ]:
df.info()

In [ ]:
df.to_parquet(parquet_data.joinpath('researchers.parquet'))

In [ ]:
file_p = "/Users/lcsanchez/Desktop/Research/researchportal.uc3m.es/parquet/researchers.parquet"
df = pd.read_parquet(file_p)
df

# 4. Tables `publications` and `researchers_publications`

In [ ]:
all_activities = list(raw_data.glob("act*"))

all_publis = []
res_publis = []

nact = 0

for file_path in tqdm(all_activities):
    if file_path.is_file():
        with file_path.open("r") as fin:
            content = fin.read()

            # Crear un objeto BeautifulSoup
            soup = BeautifulSoup(content, "html.parser")
            activity_type = soup.find(class_="display-title").get_text().strip()
            if activity_type in ['Articles', 'Book Chapters', 'Conference Contributions']:
                
                # Título
                try: 
                    title = soup.title.get_text()
                except:
                    title = None
                
                # DOI
                try:
                    doi = soup.find('a', title="Digital Object Identifier (DOI)").get_text()
                except:
                    doi = None

                # Year
                try:
                    year = int(soup.find(id='dateTimeValue-DateTimeValue-List').get_text().strip().split()[-1])
                except:
                    year = np.nan

                # Abstract
                try:
                    abstract = soup.find(id='abstract-noRangeClass-List').get_text().strip()
                except:
                    abstract = None

                # Keywords
                try:
                    keywords = soup.find(id='freetextKeyword-noRangeClass-List').get_text().strip()
                except:
                    keywords = None
                
                # Research Areas
                try:
                    research_areas_element = soup.find('h3', {'id': 'hasResearchArea'})
                    if research_areas_element:
                        research_areas_list = research_areas_element.find_next_sibling('ul').find_all('li')
                        research_areas = [area.get_text(strip=True) for area in research_areas_list]

                except:
                    research_areas = None

                # Publisher
                try:
                    if activity_type == "Articles":
                        publication_venue_element = soup.find('h3', {'id': 'hasPublicationVenue'}).find_next_sibling('ul').find('li').find('a')
                        publication_venue = publication_venue_element.get_text(strip=True) if publication_venue_element else None
                        # print("Publication Venue: ", publication_venue)

                    elif activity_type in ["Book Chapters", "Conference Contributions"]:
                        publication_venue_element = soup.find('h3', {'id': 'publisher'}).find_next_sibling('ul').find('li').find('a')
                        publication_venue = publication_venue_element.get_text(strip=True) if publication_venue_element else None
                        # print("Publisher:", publication_venue)

                except:
                    publication_venue = None
                    # print("Publisher:", publication_venue)

                # ISSN
                try:
                    issn_element = soup.find('h3', {'id': 'issn'}).find_next_sibling('ul').find('li')
                    issn_number = issn_element.get_text(strip=True) if issn_element else None
                except:
                    issn_number = None

                # EISSN
                try:
                    eissn_element = soup.find('h3', {'id': 'eissn'}).find_next_sibling('ul').find('li')
                    eissn_number = eissn_element.get_text(strip=True) if eissn_element else None
                except:
                    eissn_number = None
                    
                # ISBN
                try:
                    isbn_element = soup.find('h3', {'id': 'uc3mIsbn'}).find_next_sibling('ul').find('li')
                    isbn_number = isbn_element.get_text(strip=True) if isbn_element else None
                except:
                    isbn_number = None

                    
                # Autores
                try:
                    authors = soup.find('ul', id='relatedBy-Authorship-List').find_all('li', role="listitem")
                    for idx,el in enumerate(authors):
                        if el.find('a'):
                            res_publis.append([file_path.stem, el.find('a')['href'].strip().split('.html')[0], idx+1])
                except:
                    pass

                all_publis.append([file_path.stem, activity_type, title, abstract, keywords, research_areas, doi, year, publication_venue, issn_number, eissn_number, isbn_number])

columns = ['actID', 'ActivityType', 'Title', 'Abstract', 'Keywords', 'Research Areas', 'DOI', 'Year', 'Publisher', 'ISSN', 'EISSN', 'ISBN']
df_publis = pd.DataFrame(all_publis, columns=columns)

columns = ['actID', 'invID', 'Order']
df_res_publis = pd.DataFrame(res_publis, columns=columns)

In [ ]:
print("Number of Publications", len(df_publis))
df_publis.head(20)

In [ ]:
df_publis.info()

In [ ]:
def clean_surrogates(text):
    if text:
        try:
            # Reemplazar los caracteres de sustitución, o simplemente eliminarlos
            return text.encode('utf-16', 'surrogatepass').decode('utf-16')
            return text
        except:
            return np.nan

# Aplicar esta función a todas las columnas de texto en tu DataFrame
df_publis['Abstract'] = df_publis['Abstract'].apply(clean_surrogates)
df_publis.to_parquet(parquet_data.joinpath('publications.parquet'))

In [ ]:
print("Number of Signatures", len(df_res_publis))
df_res_publis.head()

In [ ]:
df_res_publis.info()

In [ ]:
df_res_publis.to_parquet(parquet_data.joinpath('researchers_publications.parquet'))

# 4. Table `projects`

In [ ]:
all_activities = list(raw_data.glob("act*"))

all_projects = []
res_projects = []

nact = 0

for file_path in tqdm(all_activities):
    if file_path.is_file():
        with file_path.open("r") as fin:
            content = fin.read()

            # Crear un objeto BeautifulSoup
            soup = BeautifulSoup(content, "html.parser")
            activity_type = soup.find(class_="display-title").get_text().strip()
            if activity_type == "Projects":
                # Título
                try: 
                    title = soup.title.get_text()
                except:
                    title = None

                try:
                    datetime_interval_element = soup.find('h3', {'id': 'dateTimeInterval'}).find_next_sibling('ul').find('li')
                    datetime_interval_text = datetime_interval_element.get_text(strip=True) if datetime_interval_element else None

                    # Extracting years from the text with improved regular expression
                    start_year, end_year = None, None
                    
                    if datetime_interval_text:
                        # Utilizar una expresión regular para extraer los años de inicio y fin
                        year_matches = re.findall(r'\b(\d{4})\b', datetime_interval_text)
                        start_year, end_year = map(int, year_matches) if len(year_matches) == 2 else (None, None)

                except:
                    start_year, end_year = None, None

                # Funding Entity
                try:
                    funding_entity_element = soup.find('h3', {'id': 'assignedBy'})

                    if funding_entity_element:
                        funding_entity = funding_entity_element.find_next('ul').find('a').get_text(strip=True)

                except:
                    funding_entity = None
                
                # Reference
                try:
                    reference_element = soup.find('h3', {'id': 'uc3mProjectReference'}).find_next_sibling('ul').find('li')
                    reference = reference_element.get_text(strip=True) if reference_element else None

                except:
                    reference = None
                
                # Type
                try:
                    type_element = soup.find('h3', {'id': 'uc3mProjectType'}).find_next_sibling('ul').find('li')
                    project_type = type_element.get_text(strip=True) if type_element else None
                
                except:
                    project_type = None
                
                # Role: no funciona
                try:
                    researcher_role_elements = soup.find('h3', {'id': 'relates-ResearcherRole'}).find_next_sibling('ul').find_all('li')

                    for role_element in researcher_role_elements:
                        researcher_name_element = role_element.find('a', title='name')
                        researcher_role_text = role_element.get_text()

                        if researcher_name_element:
                            researcher_id_match = re.search(r'inv(\d+)', researcher_name_element['href'])
                            researcher_id = researcher_id_match.group(0) if researcher_id_match else None
                        else:
                            researcher_id = None

                        researcher_role_match = re.compile(r'(Principal Researcher|Researcher)').search(researcher_role_text)
                        researcher_role = researcher_role_match.group(1) if researcher_role_match else None

                        res_projects.append([file_path.stem, researcher_id, researcher_role])

                except:
                    researcher_role = None
                
                all_projects.append([file_path.stem, title, start_year, end_year, funding_entity, reference, project_type])

columns = ['actID', 'Title', 'StartYear', 'EndYear', 'Funding Entity', 'Reference', 'Type']
df_projects = pd.DataFrame(all_projects, columns=columns)
# Replace None with np.nan in 'StartYear' and 'EndYear' columns
df_projects['StartYear'] = df_projects['StartYear'].replace({None: np.nan})
df_projects['EndYear'] = df_projects['EndYear'].replace({None: np.nan})

# Convert 'StartYear' and 'EndYear' columns to integers
df_projects['StartYear'] = df_projects['StartYear'].astype('Int64')
df_projects['EndYear'] = df_projects['EndYear'].astype('Int64')


columns = ['actID', 'invID', 'Role']
df_res_projects = pd.DataFrame(res_projects, columns=columns)

In [ ]:
df_projects 

In [ ]:
print("Number of Projects", len(df_projects))
df_projects.head()

In [ ]:
df_projects.info()

In [ ]:
df_projects.to_parquet(parquet_data.joinpath('projects.parquet'))

In [ ]:
df_res_projects

In [ ]:
print("Number of Signatures", len(df_res_projects))
df_res_projects.head()

In [ ]:
df_res_projects.info()

In [ ]:
df_res_projects.to_parquet(parquet_data.joinpath('researchers_projects.parquet'))